### test.csv 사용해서 최종 결과 내보기

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
train = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\train.csv", encoding="utf-8")
test = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\test.csv", encoding="utf-8")

print(f"train: {train.shape}\ntest: {test.shape}")

train: (256351, 69)
test: (90067, 68)


In [26]:
# ID 컬럼을 따로 저장
test_ids = test['ID']
y_train = train['임신 성공 여부']  # 타겟 변수

In [27]:
# selected_features에서 '임신 성공 여부'를 제외한 feature들만 선택
selected_features = [
    "시술 당시 나이", "시술 유형", "총 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수",
    "여성 주 불임 원인", "여성 부 불임 원인", "남성 주 불임 원인", "남성 부 불임 원인", "부부 주 불임 원인",
    "부부 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 남성 요인", "불명확 불임 원인",
    "불임 원인 - 자궁내막증", "불임 원인 - 자궁경부 문제", "불임 원인 - 정자 농도", "불임 원인 - 정자 형태",
    "불임 원인 - 정자 운동성", "신선 배아 사용 여부", "동결 배아 사용 여부", "해동된 배아 수", "수집된 신선 난자 수",
    "파트너 정자와 혼합된 난자 수", "이식된 배아 수"
]

# train 데이터에서 선택한 feature들과 타겟 변수 포함
train = train[selected_features]  # feature들

# test 데이터에서는 타겟 변수 없으므로 selected feature만 선택
test = test[selected_features]  # feature들


### 데이터 전처리

In [28]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# train에서 시술 유형 레이블 인코딩 (0, 1로 변환)
train['시술 유형'] = label_encoder.fit_transform(train['시술 유형'])
# test에서 시술 유형을 같은 방식으로 변환 (fit은 train에서만, transform은 test에서도)
test['시술 유형'] = label_encoder.transform(test['시술 유형'])

# DI 시술에 대해 결측치 채우기
columns_with_na = ['해동된 배아 수', '수집된 신선 난자 수', '파트너 정자와 혼합된 난자 수', '이식된 배아 수', "신선 배아 사용 여부", "동결 배아 사용 여부"]
for col in columns_with_na:
    train.loc[train['시술 유형'] == 0, col] = train.loc[train['시술 유형'] == 0, col].fillna(0)
    test.loc[test['시술 유형'] == 0, col] = test.loc[test['시술 유형'] == 0, col].fillna(0)


In [29]:
# "시술 당시 나이" 변환 함수
def convert_age(age):
    if age == "만18-34세":
        return 0
    elif age == "만35-37세":
        return 1
    elif age == "만38-39세":
        return 2
    elif age == "만40-42세":
        return 3
    elif age == "만43-44세":
        return 4
    elif age == "만45-50세":
        return 5
    else:  # "알 수 없음"
        return np.nan  # "알 수 없음"을 NaN으로 처리

# Age 칼럼에 적용
train['시술 당시 나이'] = train['시술 당시 나이'].apply(convert_age)
test['시술 당시 나이'] = test['시술 당시 나이'].apply(convert_age)

# NaN 값을 평균값으로 채우기
age_mean = train['시술 당시 나이'].mean()
train['시술 당시 나이'] = train['시술 당시 나이'].fillna(age_mean)
test['시술 당시 나이'] = test['시술 당시 나이'].fillna(age_mean)


In [30]:
# 총 시술 횟수 변환
def convert_string_to_num(count):
    mapping = {
        "0회": 0, "1회": 1, "2회": 2, "3회": 3, "4회": 4, "5회": 5, "6회 이상": 6
    }
    return mapping.get(count, None)  # 알 수 없음 처리

# 시술 횟수 처리
for column in ['총 시술 횟수', 'IVF 임신 횟수', 'IVF 출산 횟수', 'DI 임신 횟수', 'DI 출산 횟수']:
    train[column] = train[column].apply(convert_string_to_num)
    test[column] = test[column].apply(convert_string_to_num)

In [31]:
# 잘 변환된 것 확인.
print(train.head())

   시술 당시 나이  시술 유형  총 시술 횟수  IVF 임신 횟수  IVF 출산 횟수  DI 임신 횟수  DI 출산 횟수  \
0       0.0      1        0          0          0         0         0   
1       5.0      1        0          0          0         0         0   
2       0.0      1        1          0          0         0         0   
3       1.0      1        1          0          0         0         0   
4       0.0      1        0          0          0         0         0   

   여성 주 불임 원인  여성 부 불임 원인  남성 주 불임 원인  ...  불임 원인 - 자궁경부 문제  불임 원인 - 정자 농도  \
0           0           0           0  ...                0              0   
1           0           0           0  ...                0              0   
2           0           0           0  ...                0              0   
3           0           0           0  ...                0              0   
4           0           0           0  ...                0              0   

   불임 원인 - 정자 형태  불임 원인 - 정자 운동성  신선 배아 사용 여부  동결 배아 사용 여부  해동된 배아 수  \
0              0    

In [32]:
# 잘 변환된 것 확인.
print(test.head())

   시술 당시 나이  시술 유형  총 시술 횟수  IVF 임신 횟수  IVF 출산 횟수  DI 임신 횟수  DI 출산 횟수  \
0       1.0      1        1          0          0         0         0   
1       0.0      1        0          0          0         0         0   
2       3.0      1        0          0          0         0         0   
3       3.0      1        0          0          0         0         0   
4       1.0      1        1          0          0         0         0   

   여성 주 불임 원인  여성 부 불임 원인  남성 주 불임 원인  ...  불임 원인 - 자궁경부 문제  불임 원인 - 정자 농도  \
0           0           0           0  ...                0              0   
1           0           0           0  ...                0              0   
2           0           0           0  ...                0              0   
3           0           0           0  ...                0              0   
4           0           0           0  ...                0              0   

   불임 원인 - 정자 형태  불임 원인 - 정자 운동성  신선 배아 사용 여부  동결 배아 사용 여부  해동된 배아 수  \
0              0    

In [33]:
#결측치 수
train.isnull().sum()

시술 당시 나이            0
시술 유형               0
총 시술 횟수             0
IVF 임신 횟수           0
IVF 출산 횟수           0
DI 임신 횟수            0
DI 출산 횟수            0
여성 주 불임 원인          0
여성 부 불임 원인          0
남성 주 불임 원인          0
남성 부 불임 원인          0
부부 주 불임 원인          0
부부 부 불임 원인          0
불임 원인 - 난관 질환       0
불임 원인 - 배란 장애       0
불임 원인 - 남성 요인       0
불명확 불임 원인           0
불임 원인 - 자궁내막증       0
불임 원인 - 자궁경부 문제     0
불임 원인 - 정자 농도       0
불임 원인 - 정자 형태       0
불임 원인 - 정자 운동성      0
신선 배아 사용 여부         0
동결 배아 사용 여부         0
해동된 배아 수            0
수집된 신선 난자 수         0
파트너 정자와 혼합된 난자 수    0
이식된 배아 수            0
dtype: int64

In [34]:
#결측치 수
test.isnull().sum()

시술 당시 나이            0
시술 유형               0
총 시술 횟수             0
IVF 임신 횟수           0
IVF 출산 횟수           0
DI 임신 횟수            0
DI 출산 횟수            0
여성 주 불임 원인          0
여성 부 불임 원인          0
남성 주 불임 원인          0
남성 부 불임 원인          0
부부 주 불임 원인          0
부부 부 불임 원인          0
불임 원인 - 난관 질환       0
불임 원인 - 배란 장애       0
불임 원인 - 남성 요인       0
불명확 불임 원인           0
불임 원인 - 자궁내막증       0
불임 원인 - 자궁경부 문제     0
불임 원인 - 정자 농도       0
불임 원인 - 정자 형태       0
불임 원인 - 정자 운동성      0
신선 배아 사용 여부         0
동결 배아 사용 여부         0
해동된 배아 수            0
수집된 신선 난자 수         0
파트너 정자와 혼합된 난자 수    0
이식된 배아 수            0
dtype: int64

In [35]:
# 전처리 후 x_train과 x_test 업데이트
x_train = train[selected_features]
x_test = test[selected_features]

In [37]:
# 정규화.(0~1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# x_train에서 선택한 열
columns_to_scale_train = ["시술 당시 나이", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수", 
                          "이식된 배아 수", "해동된 배아 수", "수집된 신선 난자 수", "파트너 정자와 혼합된 난자 수"]

# x_test에서 선택한 열
columns_to_scale_test = ["시술 당시 나이", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수", 
                         "이식된 배아 수", "해동된 배아 수", "수집된 신선 난자 수", "파트너 정자와 혼합된 난자 수"]


# train과 test 데이터의 해당 열만 정규화
x_train[columns_to_scale_train] = scaler.fit_transform(x_train[columns_to_scale_train])
x_test[columns_to_scale_test] = scaler.transform(x_test[columns_to_scale_test])


### 모델 학습

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 로지스틱 회귀 모델 객체 생성
logreg = LogisticRegression(max_iter=1000)

# 모델 학습
logreg.fit(x_train, y_train)

# 학습된 모델로 예측
y_prob = logreg.predict_proba(x_test)
y_prob_class_1 = y_prob[:, 1]
# 예측 결과 확인
print(f"예측된 값: {y_prob_class_1}")


예측된 값: [9.98257366e-01 7.13004609e-02 9.99963096e-01 ... 1.00000000e+00
 1.55557344e-04 9.99584296e-01]


In [46]:
# 로지스틱 회귀 모델로 확률 예측

# 예측 결과를 DataFrame으로 저장 (id 컬럼은 test 데이터에서 가져옴)
result = pd.DataFrame({
    'ID': test_ids,  # test 데이터의 id를 사용
    'probability': y_prob_class_1  # 예측된 값
})

# 예측 결과를 CSV 파일로 저장
result.to_csv('prediction_results.csv', index=False, encoding='utf-8-sig')


In [47]:
# 정확도 평가 (train 데이터를 이용)
train_pred = logreg.predict(x_train)
print(f"훈련 데이터 정확도: {accuracy_score(y_train, train_pred)}")


훈련 데이터 정확도: 0.7405900503606383


다콘 결과: 0.60932

뭐가 문제일까..^^